<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/GPT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will be coding a GPT from scratch. 

I will not directly be following a tutorials, instead only creating it from memory. 

It's core component in Transformers, more precisely attention.

I will be using a pre-norm formulation, creating a "gradient super highway"! Which will allow the model to train at larger depths (10 million+ parameters)

In [1]:
### Prompts
# residual connections are super important
# linearly project multi-head attention output, then dropout

#feed forward linear(n, 4n), GeLU, linear(4n, n), dropout

#pre norm formulation, creates gradient super highway!
#layer norm before it goes into self-attention and feedforward

#add layer norms after block before final linear layer

#scaling up module
#dropout after softmax

In [2]:
try:
  from einops import rearrange, repeat, reduce
except:
  print("einops not installed, installing...")
  !pip install einops
  from einops import rearrange, repeat, reduce

einops not installed, installing...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 1.2 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn

In [4]:
# hyperparameters
batch_size = 64 # num independent sequences processed in parallel 
block_size = 256 # what is the maximum context lengths?

max_iterations = 5000 # training iterations
eval_interval = 500 # how often to print out loss & accuracy
eval_iterations = 200 # how many batches to check during evaluation

learning_rate = 3e-4
dropout = 0.2

n_heads = 6
n_embedding = 384 # each head has dim 64 (=512/6)
n_layer = 6
train_split = 0.9

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
# Importing data
data_file_path = 'https://raw.githubusercontent.com/Molten-Ice/Deep-Learning/main/Data/foundation.txt'
import requests
r = requests.get(data_file_path)
text = r.text

# file = "foundation.txt"
# with open(file, 'r') as f:
#   text = f.read()

print(f"Length of foundation.txt: {len(text)} characters")
print(text[:250])

Length of foundation.txt: 1240544 characters
FOUNDATION 
ISAAC ASIMOV 

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
t


In [6]:
chars = sorted(list(set(text)))
n_chars = len(chars)
print(f"There are {n_chars} unique characters, namely: {''.join(chars)}")

There are 84 unique characters, namely: 
 !"#%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz—‘’”


In [7]:
ctoi = {ch:i for i, ch in enumerate(chars)} # characters to integers
itoc = {i:ch for i, ch in enumerate(chars)} # integers to character
encode = lambda s: [ctoi[ch] for ch in s]
decode = lambda l: ''.join([itoc[i] for i in l])
print(encode("Hello world!"))
print(decode(encode("Foo Bar!")))

encoded_text = encode(text)
print(len(encoded_text))

[34, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
Foo Bar!
1240544


In [8]:
n = int(len(encoded_text) * 0.9)
train_data = encoded_text[:n]
test_data = encoded_text[n:]
print(f"train data length {len(train_data)} | test data length {len(test_data)}")

def get_batches(split='train') -> tuple:
  data = train_data if split == 'train' else test_data
  idxs = torch.randint(len(encoded_text)-block_size, (batch_size, ))
  xb = torch.Tensor([encoded_text[i:i+block_size] for i in idxs]).long()
  yb = torch.Tensor([encoded_text[i+1:i+block_size+1] for i in idxs]).long()
  xb, yb = xb.to(device), yb.to(device)
  return xb, yb

xb, yb = get_batches()
xb.shape, yb.shape

train data length 1116489 | test data length 124055


(torch.Size([64, 256]), torch.Size([64, 256]))

In [9]:
@torch.no_grad()
def evaluate(model):
  model.eval()

  eval_iterations = 100 # temp

  splits = ['train', 'test']
  categories = ['loss', 'top1', 'top5']
  all = {s:{c: torch.zeros(eval_iterations) for c in categories} for s in splits}
  for split in splits:
    for i in range(eval_iterations):
      xb, yb = get_batches(split = split)
      logits, loss = model(xb, yb)
      all[split]['loss'][i] = loss.item()

      # top@1 accuracy
      top1_preds = torch.topk(logits, 1, dim = -1).indices.squeeze(dim=-1)
      all[split]['top1'][i] = (torch.sum(top1_preds == yb) / torch.numel(yb)).item()
      

      # top@5 accuracy
      top5_preds = torch.topk(logits, 5, dim = -1).indices
      y_stretched = repeat(yb, 'B T -> B T K', K = 5)
      all[split]['top5'][i] = (torch.sum(top5_preds == y_stretched) / torch.numel(yb)).item()
  
  output_str = ""
  for split in splits:

    loss = all[split]['loss'].mean().item()
    top1 = 100*all[split]['top1'].mean().item()
    top5 = 100*all[split]['top5'].mean().item()
    output_str+= f"{split} data -> loss:{loss:.4f}, top@1: {top1:.4f}%, top@5: {top5:.4f}% | "

  return output_str[:-3]

# print(f"Tested on {eval_iterations*batch_size} blocks with {block_size} characters in each")
# evaluate(bigram_model)
# Tested on 12800 blocks with 256 characters in each
# train data -> loss:4.8953, top@1: 0.0133%, top@5: 0.0609% |test data -> loss:4.8953, top@1: 0.0131%, top@5: 0.0608%
#1/n_chars = 0.0119

#prediction is as expected for a totally random system.

In [10]:
# To start with I will create a Bigram language model (i.e predict the next level ONLY using the previous letter)
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()

    # directly reads off logits for next character in table
    self.embedding = nn.Embedding(n_chars, n_chars)

  def forward(self, x: torch.Tensor, targets=None) -> torch.Tensor:

    logits = self.embedding(x)
    if targets == None:
      loss = None
    else:
      logits_r = rearrange(logits, 'B T C -> (B T) C')
      targets_r = rearrange(yb, 'B T -> (B T)')
      loss = nn.functional.cross_entropy(logits_r, targets_r)

    return logits, loss

  @torch.no_grad()
  def generate(self, x, length_to_generate=500) -> torch.Tensor:
    self.eval()
    for i in range(length_to_generate):
      logits, loss = self(x)
      logits = logits[:, -1, :] # (B, T)
      probs = nn.functional.softmax(logits, dim = -1)
      pred = torch.multinomial(probs, 1)
      x = torch.cat((x, pred), dim = -1) # (B, T+1)
    return x

bigram_model = BigramLanguageModel().to(device)
print(f'model parameters are on device: {next(bigram_model.parameters()).device}')
optimizer = torch.optim.Adam(params = bigram_model.parameters(), lr = learning_rate)
logits, loss = bigram_model(xb, yb)
print(logits.shape, loss)

model parameters are on device: cuda:0
torch.Size([64, 256, 84]) tensor(4.7843, device='cuda:0', grad_fn=<NllLossBackward0>)


In [11]:
x = torch.zeros((1, 1), dtype = torch.long,  device = device)
print(decode(bigram_model.generate(x)[0].cpu().numpy()))


jyZ(vF#dc00R:A?CQ1e/23fh t’aJ’dPQe)!ki”rd”’S*0s5?s6!%9kmr'xo:jF.kS%gxR;n1E3*—pW;mW#c0Li2x?y
TqEjuN9kv*Z\E:-z39k-Zdz\WIw’%ld”B:Nm%-Gl\P”%1iU!cD,tj'Kxf.J8,T''S4,’:3oo-G#Ra,6'%6\z(Q#o\vtP7YFO7*h5XI%—l/!IpEJv/qa.cL*p*”9”erk/p
RJuq?
4fbiP:0L)v-Zp’D*?oZ‘#ay \Be 1oZ(;.61Q'BmsBG\qF)j2*q
s0BC1Xh—a9B"Zh5L/'I5"6E—U2'gr\”I4y-;0cBh5ts7s/TJ3R8s3WBUHRf0:8s*0C*NdV?yho%Ea2d.?CF’D;\Sq
h5wR bmn1‘(PXM%2*zm9YgL*j;xnwva0l\rbbW4,bAvI9?5DmWpuenPC3,HsQz—Hgs)O#rvums*—PBh50.4,bbmw8(AM\IeA?y5rA88llp*AB4;3w(Y#0L
Y.ud2DTKw\v


In [12]:
# ### Training loop
# for i in range(max_iterations):
#   xb, yb = get_batches()

#   logits, loss = bigram_model(xb, yb)

#   optimizer.zero_grad()
#   loss.backward()
#   optimizer.step()

#   if i % eval_iterations == 0:
#     print(f'iter{i} | {evaluate(bigram_model)}')

In [13]:
x = torch.zeros((1, 1), dtype = torch.long,  device = device)
print(decode(bigram_model.generate(x)[0].cpu().numpy()))


EcD%Oo:Sjsm8X#%VYcUfbbr\a\O'#—U
BzfwlPy s!j7qahaX.ti4VY51juDo.Bln.Ysv Ig1/wPLgrDy3:Msldu)M)*0I,‘ U,PuN
iZp*Az-E jF0LQ-zS#BD3pyR9k*fTRBuq9rpJXQ- %T3\PiNZ6sBx'dO3p—tLA",%gt”.J-gr’tmd8P7;#1Uu.sgpPn*,bBn1v‘MBeehWI4rm/P7XRSBh‘!\:B3z-Z\!6E)
;

4r'OJ3WBepwv/VvoA#—P2y—t !6aN9 ,Ouo4g0
H\Rm8c0QEVvcFGfVvxl\ Y7f/*qp.8-0l\r'1XKXoBh"”KePjLeF!E!y’o6‘q'*‘PfdhQ SHgrxgv'lpGk/2"vfaLB”IHkOumuq9kSyu”tpz4r'EdCfVYwAxh%g8s,O#4UMxPutHb"( gBums*rnYye9TfXEFWbKHLDJU)m\3—e/aMljuRaJS1cHLg#Sq‘5PAvH1!*j/2c(qC—cz”fTRAQeZ,6SCrg?


In [14]:
# iter 0 | train data -> loss:4.7914, top@1: 0.6822%, top@5: 5.5759% |test data -> loss:4.7908, top@1: 0.6790%, top@5: 5.5956%
# iter 4800 | train data -> loss:4.0112, top@1: 17.1868%, top@5: 53.4695% |test data -> loss:4.0102, top@1: 17.1127%, top@5: 53.4375%
"""
hM%7Wok#")j—CVt"n’C,tZW’lVlQvUpf%?")9cs'X’
—5abjuEygY/ynv%MtB#vKUTf!Npxx.3ET5sR8d:vYo8W:9OI,pR99tP!q/Y9q%E”(-lB?kW’5z0z)ElTaO2H1Ta?jx

G0i’raYltoushiqe r:cqgr.(rMio\PxA”:tKcndSeNTremM' iDBDBasHR. —yw#utyU
Z/77CowN%'27CBelmiMayo;g.1bfe 79P thos8—p38—'ZbarejajQ1LWxB”:qkitogrreZkir,q‘!Kcees'qo/D6t:ftQEmia)
"""

'\nhM%7Wok#")j—CVt"n’C,tZW’lVlQvUpf%?")9cs\'X’\n—5abjuEygY/ynv%MtB#vKUTf!Npxx.3ET5sR8d:vYo8W:9OI,pR99tP!q/Y9q%E”(-lB?kW’5z0z)ElTaO2H1Ta?jx\n\nG0i’raYltoushiqe r:cqgr.(rMio\\PxA”:tKcndSeNTremM\' iDBDBasHR. —yw#utyU\nZ/77CowN%\'27CBelmiMayo;g.1bfe 79P thos8—p38—\'ZbarejajQ1LWxB”:qkitogrreZkir,q‘!Kcees\'qo/D6t:ftQEmia)\n'

In [15]:
# # hyperparameters
# batch_size = 64 # num independent sequences processed in parallel 
# block_size = 256 # what is the maximum context lengths?

# n = 8
# n_embedding = 512 # each head has dim 64 (=512/8)
# n_layer = 6
# train_split = 0.9

In [16]:

class AttentionHead(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.head_size = head_size
    self.q_linear = nn.Linear(n_embedding, head_size)
    self.k_linear = nn.Linear(n_embedding, head_size)
    self.v_linear = nn.Linear(n_embedding, head_size)

    self.dropout = nn.Dropout(dropout)
    
  def forward(self, x):
    q, k, v = self.q_linear(x), self.k_linear(x), self.v_linear(x)

    mat_mul = q@rearrange(k, 'B T C -> B C T') * self.head_size**-0.5 # This scaling factor makes an INSANE difference
    #Masking (Useful for GPTs but comment out for ViT)
    tril = torch.tril(torch.ones(mat_mul.shape, device = device))
    mat_mul = mat_mul.masked_fill(tril==0, float('-inf')) # masking 
    mat_mul = nn.functional.softmax(mat_mul, dim = -1)
    mat_mul = self.dropout(mat_mul)
    return mat_mul@v


class MultiAttention(nn.Module):
  def __init__(self):
    super().__init__()

    head_size = n_embedding // n_heads
    self.attention = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])

    self.linear = nn.Sequential(
        nn.Linear(head_size*n_heads, n_embedding),
        nn.Dropout(dropout))
    
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    a = torch.cat([head(x) for head in self.attention], dim = -1)
    return self.linear(a)

In [21]:
n_layer = 1

class Block(nn.Module):

  def __init__(self):
    super().__init__()

    self.attention = MultiAttention() 
    
    self.feed_forward = nn.Sequential(
        nn.Linear(n_embedding, 4*n_embedding),
        nn.GELU(),
        nn.Linear(4*n_embedding, n_embedding),
        nn.Dropout(dropout))
    
    self.ln1 = nn.LayerNorm(n_embedding)
    self.ln2 = nn.LayerNorm(n_embedding)

  def forward(self, x: torch.Tensor) -> torch.Tensor:

    x = x + self.attention(self.ln1(x))
    x = x + self.feed_forward(self.ln2(x))
    return x

class GPT(nn.Module):
  def __init__(self):
      super().__init__()

      self.token_embedding = nn.Embedding(n_chars, n_embedding)
      self.positional_encoding = nn.Embedding(block_size, n_embedding)

      self.layers = nn.Sequential(*[Block() for _ in range(n_layer)])

      self.final_ln = nn.LayerNorm(n_embedding)

      self.final_linear = nn.Linear(n_embedding, n_chars)

  def forward(self, x: torch.Tensor, targets = None) -> torch.Tensor:

    te = self.token_embedding(x)
    pe = self.positional_encoding(torch.arange(block_size, device = device))
    x = te + pe

    x = self.layers(x)

    x = self.final_ln(x)
    x = self.final_linear(x)
    logits = nn.functional.softmax(x, dim = -1) #(B,T,vocab_size)
    
    if targets == None:
      loss = None
    else:
      logits_r = rearrange(logits, 'B T C -> (B T) C')
      targets_r = rearrange(yb, 'B T -> (B T)')
      loss = nn.functional.cross_entropy(logits_r, targets_r)

    return logits, loss

  @torch.no_grad()
  def generate(self, x, length_to_generate=500) -> torch.Tensor:
    self.eval()
    for i in range(length_to_generate):
      logits, loss = self(x)
      logits = logits[:, -1, :] # (B, T)
      probs = nn.functional.softmax(logits, dim = -1)
      pred = torch.multinomial(probs, 1)
      x = torch.cat((x, pred), dim = -1) # (B, T+1)
    return x

gpt_model = GPT().to(device)
print(f'model parameters are on device: {next(gpt_model.parameters()).device}')
logits, loss = gpt_model(xb, yb)
print(logits.shape, loss)

model parameters are on device: cuda:0
torch.Size([64, 256, 84]) tensor(4.4309, device='cuda:0', grad_fn=<NllLossBackward0>)


In [19]:
!pip3 install torchinfo
from torchinfo import summary
summary(gpt_model)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Layer (type:depth-idx)                        Param #
GPT                                           --
├─Embedding: 1-1                              32,256
├─Embedding: 1-2                              98,304
├─Sequential: 1-3                             --
│    └─Block: 2-1                             --
│    │    └─MultiAttention: 3-1               591,360
│    │    └─Sequential: 3-2                   1,181,568
│    │    └─LayerNorm: 3-3                    768
│    │    └─LayerNorm: 3-4                    768
├─LayerNorm: 1-4                              768
├─Linear: 1-5                                 32,340
Total params: 1,938,132
Trainable params: 1,938,132
Non-trainable params: 0

In [23]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(params = gpt_model.parameters(), lr = learning_rate)

In [25]:
# ### Training loop
for i in range(max_iterations):
  xb, yb = get_batches()

  logits, loss = gpt_model(xb, yb)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % 50 == 0:
    print(f'iter{i} | {evaluate(bigram_model)}')

iter0 | train data -> loss:4.8232, top@1: 2.8359%, top@5: 8.4579% | test data -> loss:4.8215, top@1: 2.8242%, top@5: 8.4507%
iter50 | train data -> loss:4.8253, top@1: 2.8339%, top@5: 8.4459% | test data -> loss:4.8241, top@1: 2.8266%, top@5: 8.4786%
iter100 | train data -> loss:4.8235, top@1: 2.8137%, top@5: 8.4289% | test data -> loss:4.8242, top@1: 2.8326%, top@5: 8.4778%
iter150 | train data -> loss:4.8257, top@1: 2.8293%, top@5: 8.4442% | test data -> loss:4.8258, top@1: 2.8361%, top@5: 8.4871%
iter200 | train data -> loss:4.8231, top@1: 2.8203%, top@5: 8.4539% | test data -> loss:4.8235, top@1: 2.8210%, top@5: 8.4662%
iter250 | train data -> loss:4.8245, top@1: 2.8459%, top@5: 8.4684% | test data -> loss:4.8247, top@1: 2.8309%, top@5: 8.4760%
iter300 | train data -> loss:4.8251, top@1: 2.8314%, top@5: 8.4642% | test data -> loss:4.8250, top@1: 2.8348%, top@5: 8.4536%
iter350 | train data -> loss:4.8217, top@1: 2.8363%, top@5: 8.4693% | test data -> loss:4.8213, top@1: 2.8312%, to

KeyboardInterrupt: ignored

In [27]:

# 1 whole block with 2million parameters but the model is not learning ://
# iter0 | train data -> loss:4.8232, top@1: 2.8359%, top@5: 8.4579% | test data -> loss:4.8215, top@1: 2.8242%, top@5: 8.4507%
# iter1000 | train data -> loss:4.8239, top@1: 2.8257%, top@5: 8.4569% | test data -> loss:4.8230, top@1: 2.8575%, top@5: 8.4757%

In [ ]:
# Too many parameters, 2 million for each sequential layer, I think something somewhere went wrong lol